In [1]:
import pandas as pd
import re
import sys
from PyPDF2 import PdfFileReader
import docx2txt
import openpyxl
import gspread
import df2gspread as d2g

In [2]:
#Extract pdf text
file = open('/Users/hector/Desktop/pau/cvz.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " " #empty variable for the extracted text
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

                  COUNTY OF ZAVALA 293RD DISTRICT COURT CIVIL PENDING CASES AS OF 01/21/2022                                           RAN ON JANUARY 21, 2022 10:09am PAGE: 1        CAUSE #             FILE DATE CAUSE OF ACTION           DOCKT DATE  DOCKET TYPE       ANS FILED  CR CASE #PLAINTIFF NAME                PLAINTIFF ATTORNEY        DEFENDANT NAME                DEFENDANT ATTORNEY        00-01-10044-ZCV    03/24/2010 PET TO MODIFY                                                              RIZO, JESUS                   NIETO, PEDRO G.           CORREA, DAGOBERTO                                          -------------------------------------------------------------------------------------------------------------        00-04-0816-TX      04/14/2000 TAX SUIT                                                                   UVALDE COUNTY APPRAISAL DIS                             GOMEZ, MARGARITO                                                                                      

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
#regex - finds and extracts cause_numbers, file_date, cause_action(24 character), docket_date, docket-type, plaintiff (21 character)
finds_cause_numbers = re.findall(r'(\d{2}-\d{2}-\d{5}-\w*|\d*-\d*-\d*-\w*|\d*-\w*)\s*(\d{2}/\d{2}/\d{4})\s*(\D.{23})\s*(\d{2}/\d{2}/\d{4}|[ ]{0,1})(\D.[M$][O$][T$][I$].{,12}|\D.[W$][O$][N$]..{,12}|\D.[T$][R$][I$]..{,12}|\D.[J$][U$][R$]..{,12}|\D.[S$][T$][A$]..{,12}|\D.[H$][E$][A$]..{,12}|\D.[P$][R$][E$]..{,12}|\D.[E$][N$][T$]..{,12}|\D.[P$][E$][T$]..{,12}|\D.[F$][I$][N$]..{,12}|\D.[C$][O$][M$]..{,12}|\D.[P$][L$][E$]..{,12}|\D.[N$][O$][T$]..{,12}|[ ]{0,1})\s(\D.{,20})', content)#pattern for cause number
finds_cause_numbers


[('00-01-10044-ZCV',
  '03/24/2010',
  'PET TO MODIFY           ',
  '',
  '',
  'RIZO, JESUS          '),
 ('00-04-0816-TX',
  '04/14/2000',
  'TAX SUIT                ',
  '',
  '',
  'UVALDE COUNTY APPRAIS'),
 ('00-04-10094-ZCV',
  '04/13/2011',
  'AG PET AFFECTING PARENT-',
  '',
  '',
  'CORTINAS, MARIA ELENA'),
 ('00-04-10098-ZCV',
  '02/08/2017',
  'AG PET AFFECTING PARENT-',
  '',
  '',
  'MARTINEZ, ROSANNA    '),
 ('00-04-10100-ZCV',
  '07/13/2017',
  'AG SUIT AFFECTING PARENT',
  '',
  '',
  'OCHOA, PRISCILLA     '),
 ('00-05-10130-ZCV',
  '10/03/2016',
  'AG PET TO ESTAB PARENT C',
  '',
  '',
  'DELGADO, GLORIA      '),
 ('01-03-10270-ZCV',
  '03/03/2010',
  'PET TO MODIFY           ',
  '',
  '',
  'MENDOZA, GABRIEL     '),
 ('01-10-10362-ZCV',
  '06/18/2014',
  'AG PETITION TO MODIFY   ',
  '',
  '',
  'TORRES, ANGELICA M   '),
 ('02-03-10446-ZCV',
  '07/27/2017',
  'AG PET AFFECTING PARENT-',
  '',
  '',
  'MELENDREZ, MAGGIE    '),
 ('02-06-10480-ZCV',
  '10/26/2021',
  

In [6]:
pending_cause_number_df = pd.DataFrame(finds_cause_numbers, columns = ['cause_number', 'file_date', 'cause_of_action', 'docket_date', 'docket_type', "plaintiff"])#converts list into df
pending_cause_number_df


,cause_number,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,00-01-10044-ZCV,03/24/2010,PET TO MODIFY,,,"RIZO, JESUS"
1,00-04-0816-TX,04/14/2000,TAX SUIT,,,UVALDE COUNTY APPRAIS
2,00-04-10094-ZCV,04/13/2011,AG PET AFFECTING PARENT-,,,"CORTINAS, MARIA ELENA"
3,00-04-10098-ZCV,02/08/2017,AG PET AFFECTING PARENT-,,,"MARTINEZ, ROSANNA"
4,00-04-10100-ZCV,07/13/2017,AG SUIT AFFECTING PARENT,,,"OCHOA, PRISCILLA"
...,...,...,...,...,...,...
771,96-09-09420-ZCV,08/27/2015,AG PET TO ESTAB PARENT C,,,"MATA, MARIVEL"
772,97-06-09578-ZCV,06/12/2008,PET TO MODIFY,,,"CISNEROS, ROSA LAURA"
773,98-04-09724-ZCV,06/29/2011,AG PET AFFECTING PARENT-,,,"VICTORINO, GUADALUPE"
774,98-12-09860-ZCV,07/09/2021,DIVORCE W/CHILDREN W/WAI,,,"OCANAS, VERONICA V."


In [ ]:
#df.to_csv('/Users/hector/Desktop/pau/cause.csv')#writes to csv

In [ ]:
#I used the merge function to add the previous pending cases with the new one that contained file_date, cause_of_action, and so forth.
#It used once to gather both data sets. Now that the dataset is one we will go back to using the .append function.
#df = civil_pending_notes.merge(pending_cause_number_df, how = 'outer', on = 'cause_number')
#df


In [5]:
#opens the google sheet of pending case notes
    #sets the json to service account path
json_path = gspread.service_account(filename = '/Users/hector/codeup-data-science/293pending_cases/pending_cases.json')
    #opens the google sheet by key found in the address
opens_civil_pending_gs = json_path.open_by_key('1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk')
    #pulls the data from the google worksheet (civil_pending_notes tab)
civil_pending_notes_tab = opens_civil_pending_gs.get_worksheet(0)
    #puts the data from the google sheet and puts it into a dataframe
civil_pending_notes = pd.DataFrame(civil_pending_notes_tab.get_all_records())
civil_pending_notes.shape

(2977, 11)

In [8]:
#adds both lists together in order to search for dups later
df = civil_pending_notes.append(pd.DataFrame(pending_cause_number_df), ignore_index=True)
df


,cause_number,notes,disposed,dispose_date,on_track,notes_lupita,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,1,(Look for this case under Plaintiff name G and...,,,,,,,,,
1,2,Case does not have any information. Please dis...,,,,,,,,,
2,6972,DISPOSED,TRUE,3/30/2011,,,,,,,
3,7038,DISPOSED,TRUE,2/11/1999,,,,,,,
4,8672,DISPOSED,TRUE,11/14/2005,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
3748,96-09-09420-ZCV,NaN,NaN,NaN,NaN,NaN,08/27/2015,AG PET TO ESTAB PARENT C,,,"MATA, MARIVEL"
3749,97-06-09578-ZCV,NaN,NaN,NaN,NaN,NaN,06/12/2008,PET TO MODIFY,,,"CISNEROS, ROSA LAURA"
3750,98-04-09724-ZCV,NaN,NaN,NaN,NaN,NaN,06/29/2011,AG PET AFFECTING PARENT-,,,"VICTORINO, GUADALUPE"
3751,98-12-09860-ZCV,NaN,NaN,NaN,NaN,NaN,07/09/2021,DIVORCE W/CHILDREN W/WAI,,,"OCANAS, VERONICA V."


In [ ]:
#df.to_csv('/Users/hector/Desktop/pau/cause.csv')#writes to csv

In [9]:
    #drops the duplicated cause numbers and reindexes the dataframe
    #resets the index and drops the output index
    #fills in the na with an empty space to avoid error
df = df.drop_duplicates('cause_number').reset_index(drop=True).fillna(' ')
df

,cause_number,notes,disposed,dispose_date,on_track,notes_lupita,file_date,cause_of_action,docket_date,docket_type,plaintiff
0,1,(Look for this case under Plaintiff name G and...,,,,,,,,,
1,2,Case does not have any information. Please dis...,,,,,,,,,
2,6972,DISPOSED,TRUE,3/30/2011,,,,,,,
3,7038,DISPOSED,TRUE,2/11/1999,,,,,,,
4,8672,DISPOSED,TRUE,11/14/2005,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
3748,96-09-09420-ZCV,,,,,,08/27/2015,AG PET TO ESTAB PARENT C,,,"MATA, MARIVEL"
3749,97-06-09578-ZCV,,,,,,06/12/2008,PET TO MODIFY,,,"CISNEROS, ROSA LAURA"
3750,98-04-09724-ZCV,,,,,,06/29/2011,AG PET AFFECTING PARENT-,,,"VICTORINO, GUADALUPE"
3751,98-12-09860-ZCV,,,,,,07/09/2021,DIVORCE W/CHILDREN W/WAI,,,"OCANAS, VERONICA V."


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3753 entries, 0 to 3752
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cause_number     3753 non-null   object
 1   notes            3753 non-null   object
 2   disposed         3753 non-null   object
 3   dispose_date     3753 non-null   object
 4   on_track         3753 non-null   object
 5   notes_lupita     3753 non-null   object
 6   file_date        3753 non-null   object
 7   cause_of_action  3753 non-null   object
 8   docket_date      3753 non-null   object
 9   docket_type      3753 non-null   object
 10  plaintiff        3753 non-null   object
dtypes: object(11)
memory usage: 322.6+ KB


In [ ]:
(df['notes'].values == ' ').sum()




In [ ]:
df.to_csv('/Users/hector/Desktop/pau/causenew.csv')#writes to csv

In [ ]:
total = df.cause_number.count()
total

In [ ]:
disposed = (df['disposed']).value_counts()['TRUE']
disposed

In [ ]:
pending = total - disposed
pending

In [ ]:
#civil_pending_notes_tab.clear()

In [ ]:
#civil_pending_notes_tab.update([df.columns.values.tolist()] + df.values.tolist())

# SCRATCH code

In [ ]:
pending_cause_number_df

In [ ]:
#adds both lists together in order to search for dups later
df = civil_pending_notes.append(pd.DataFrame(pending_cause_number_df, columns=['cause_number']), ignore_index=True)
df

In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates('cause_number').fillna(' ').reset_index(drop=True)
df

In [ ]:
# add a sheet with 20 rows and 2 columns
opens_civil_pending_gs.add_worksheet(rows=2000,cols=10,title='new')

# get the instance of the second sheet
civil_pending_notes = opens_civil_pending_gs.get_worksheet(1)

In [ ]:
    #updates the google sheet with the new list of pending cases
civil_pending_notes.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
civil_pending_notes

In [ ]:
rangeAll = '{0}!A1:Z'.format( sheetName )
body = {}
resultClear = service.spreadsheets( ).batchUpdate().values( ).clear( spreadsheetId=spreadsheet_id, range=rangeAll,
                                                       body=body ).execute( )

In [ ]:
gc = gspread.service_account(filename = '/Users/hector/Desktop/pau/pending_cases.json')
sht1 = gc.open_by_key('1b3fmZrbfwZWMvu4kUGJSSGsp61utlE0Ny-ebozZ5aBk')


In [ ]:
worksheet = sht1.get_worksheet(0)

In [ ]:

civil_pending_notes = pd.DataFrame(worksheet.get_all_records())


In [ ]:
#civil_pending_notes = pd.read_excel('civil_pending.xlsx')
civil_pending_notes


In [ ]:
df = df.fillna(' ')
df

In [ ]:
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_cause_numbers.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('yesterdays_pending.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts cause numbers from the previous day's pending report
y_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
yesterday_df = pd.DataFrame(y_causenum, columns = ['cause_number'])#puts it into a df
yesterday_df.shape

In [ ]:
#adds both lists together in order to search for dups later
df = todays_df.append(pd.DataFrame(y_causenum, columns=['cause_number']), ignore_index=True)
df.shape #shape of total rows


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates()
df.shape

In [ ]:
df

In [ ]:
#prints to excel
#add notes to this excel file
df.to_excel('pending_causes.xlsx', sheet_name='pending_cause_numbers')  

#New version of this will be a google sheet

In [ ]:
#reads in the most recent excel file that has notes ie 'latest_df'
latest_df = pd.read_excel('pending_causes.xlsx', index_col=0)#index_col = 0 so that it does not add in another indexed col
#this will read in a google sheet that has the notes in it


In [ ]:
latest_df

In [ ]:
#opens and reads the previous days pending report i.e. 'yesterdays_pending.pdf'
file = open('7_pen.pdf', 'rb') #opens pdf
pdfReader = PdfFileReader(file) #reads pdf
count = pdfReader.numPages #counts the number of pages
content = " "
for i in range(count): #for loop to extract text from all pages
    page = pdfReader.getPage(i) #gets page numbers
    content += page.extractText() #extracts text from iterated pages
    print(content)#, file = open('yesterdays_causes.txt', 'a')) #prints cause numbers to file. I did this because the extracted texts was seperated into lists by pages.
    
    

In [ ]:
#extracts new day report cause numbers to be compared to latest report
t_causenum = re.findall(r'\d{2}-\d{2}-\d{5}-\w*', content)
todays_df = pd.DataFrame(t_causenum, columns = ['cause_number'])

In [ ]:
todays_df

In [ ]:
#takes the new day report and adds it to the latest report for comparison
df = latest_df.append(pd.DataFrame(todays_df, columns=['cause_number']), ignore_index=True) #adds both lists together in order to search for dups later
df


In [ ]:
#drops the duplicated cause numbers
df = df.drop_duplicates() 
df
###Needs to be reindexes!!

In [ ]:
#prints back to excel
#update this file
df.to_excel("_pending_causes.xlsx", sheet_name='pending_cause_numbers')  

#New version of this will go back to the google sheet

In [ ]:
#duplicated=df[df.duplicated()] #finds all duplicates
#duplicated.shape #count of rows and columns of the duplicated cause numbers

In [ ]:
#shows ONLY the duplicates
#duplicated.sort_values('cause_number')